In [0]:
DROP TABLE IF EXISTS gap_catalog.log.performance_test_log;
CREATE TABLE IF NOT EXISTS gap_catalog.log.performance_test_log (
    test_case_name STRING NOT NULL COMMENT 'Name of the test case being executed',
    step_name STRING COMMENT 'Name of the specific step within the test case',
    start_time TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP() COMMENT 'Start time of the test execution',
    end_time TIMESTAMP COMMENT 'End time of the test execution',
    compute_size STRING COMMENT 'Size of the compute cluster used for testing',
    started_by STRING NOT NULL COMMENT 'User who initiated the test',
    status STRING NOT NULL COMMENT 'Status of the test (RUNNING, COMPLETED, FAILED, etc.)',
    process_key STRING COMMENT 'Unique identifier for the test process',
    p_load_date DATE COMMENT 'Load date parameter for the test execution',
    target_table STRING COMMENT 'Name of the target table being processed',
    target_insert_count BIGINT COMMENT 'Number of records inserted into target table',
    target_update_count BIGINT COMMENT 'Number of records updated in target table',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record creation timestamp',
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record last update timestamp'
)
USING DELTA
TBLPROPERTIES (
    'delta.feature.allowColumnDefaults' = 'supported',
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
)
COMMENT 'Performance testing log table to track test executions with detailed step-level metrics and target table processing counts'
;

CREATE OR REPLACE PROCEDURE gap_catalog.log.start_performance_test(
    IN test_case_name STRING,
    IN step_name STRING,
    IN compute_size STRING,
    IN started_by STRING,
    IN process_key STRING,
    IN p_load_date DATE DEFAULT NULL
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the start of a performance test with step details'
AS
BEGIN
    INSERT INTO gap_catalog.log.performance_test_log (
        test_case_name,
        step_name,
        start_time,
        compute_size,
        started_by,
        status,
        process_key,
        p_load_date,
        created_at,
        updated_at
    )
    VALUES (
        test_case_name,
        step_name,
        CURRENT_TIMESTAMP(),
        compute_size,
        started_by,
        'RUNNING',
        process_key,
        p_load_date,
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    );
END;

-- Updated procedure to complete a performance test log entry
CREATE OR REPLACE PROCEDURE gap_catalog.log.complete_performance_test(
    IN process_key STRING,
    IN status STRING,
    IN target_table STRING DEFAULT NULL,
    IN target_insert_count BIGINT DEFAULT NULL,
    IN target_update_count BIGINT DEFAULT NULL
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the completion of a performance test with target table metrics'
AS
BEGIN
    UPDATE gap_catalog.log.performance_test_log
    SET 
        end_time = CURRENT_TIMESTAMP(),
        status = status,
        target_table = target_table,
        target_insert_count = target_insert_count,
        target_update_count = target_update_count,
        updated_at = CURRENT_TIMESTAMP()
    WHERE process_key = process_key
      AND end_time IS NULL;
    
    SELECT CONCAT('Performance test completed with status: ', status, 
                  ' for process key: ', process_key,
                  CASE WHEN target_table IS NOT NULL THEN CONCAT(' - Target: ', target_table) ELSE '' END) as message;
END;

/*
-- Example 1: Start a performance test with step details
CALL gap_catalog.log.start_performance_test(
    test_case_name => 'ADS_RDS_Q',
    step_name => 'Extract_Data',
    compute_size => '2XS',
    started_by => 'FilipOliva',
    process_key => '889',
    p_load_date => '2025-09-24'
);

-- Example 2: Complete a performance test with target metrics
CALL gap_catalog.log.complete_performance_test(
    process_key => '889',
    status => 'COMPLETED',
    target_table => 'ads_rds_event_types_mep',
    target_insert_count => 50000,
    target_update_count => 5000
);
*/

-- Updated query to view performance test logs
SELECT 
    test_case_name,
    step_name,
    start_time,
    end_time,
    CASE 
        WHEN end_time IS NOT NULL THEN 
            CAST((UNIX_TIMESTAMP(end_time) - UNIX_TIMESTAMP(start_time)) AS STRING) || ' seconds'
        ELSE 'Still running'
    END as duration,
    compute_size,
    started_by,
    status,
    process_key,
    p_load_date,
    target_table,
    target_insert_count,
    target_update_count,
    CASE 
        WHEN target_insert_count IS NOT NULL AND target_update_count IS NOT NULL 
        THEN target_insert_count + target_update_count 
        ELSE NULL 
    END as total_target_records,
    created_at
FROM gap_catalog.log.performance_test_log
ORDER BY created_at DESC;